In [8]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('smallcap.csv')  # Make sure to replace '/path/to/your/smallcap.csv' with the actual file path

# Extract the tickers into a list
tickers = df['Symbol'].tolist()

# Now, 'tickers' contains all the tickers from the "Symbol" column of your CSV file
print(tickers)


['BAER', 'PNRG', 'EACO', 'FATBB', 'WEBC', 'RMGC', 'DIT', 'GATE', 'COOL', 'BLEU', 'CRAWA', 'PTWO', 'MRTI', 'SEPA', 'UTGN', 'ESAC', 'OSI', 'BLUA.U', 'FOTB', 'ALTU', 'ASCB', 'HEOL', 'BACA', 'VSTE', 'LINK', 'IRAA', 'GGAUF', 'GGAAF', 'INTE', 'MAQC', 'BIOQ', 'RDIB', 'IOR', 'VBFC', 'AIB', 'KANP', 'JANL', 'BDL', 'KWIK', 'CIBY', 'INTG', 'BIMI', 'FFWC', 'YOTA', 'DHAC', 'ADRT', 'CTNT', 'BSPA', 'VTVT', 'NCSM', 'WAVS', 'ZENV', 'BSHI', 'RAND', 'AVHIU', 'GSDI', 'SVT', 'MPAD', 'OSBK', 'BTQNF', 'MEDS', 'NSYS', 'FATH', 'PBM', 'RSRV', 'AMEN', 'LOGN', 'PCLB', 'WHLM', 'SSBP', 'MBBC', 'SPND', 'PIXY', 'MSS', 'ERKH', 'ELLH', 'AMFC', 'IMTH', 'FRFC', 'HCGI', 'AOREF', 'GVFF', 'UNTN', 'CVR', 'MINM', 'PGNT', 'LYT', 'GOVB', 'CIBN', 'NUKK', 'CCFC', 'VCNX', 'SRNN', 'LDWY', 'SCBS', 'DMSL', 'SMTK', 'SKAS', 'EZFL', 'LCTC', 'SVIN', 'LCFY', 'FXBY', 'NEXI', 'MICS', 'CAJTF', 'KAVL', 'SIGY', 'BAOS', 'TTNP', 'SUIC', 'VRPX', 'ZUKI', 'HSDT', 'UTRS', 'YOSH', 'AVTX', 'CETXP', 'CPMV', 'CETX', 'ABBB', 'FNCH', 'BPTH', 'SASI', 'SHMY'

In [113]:
import requests
from alpha_vantage.timeseries import TimeSeries
import pandas as pd
from datetime import datetime, timedelta


# Replace 'YOUR_API_KEY' with your actual Alpha Vantage API key
api_key = 'HGLDEL3XDK78QXPN'
symbol = 'IBM'  # Example symbol
interval = '5min'  # Data interval
outputsize = 'full'  # Get the full intraday data for a specific month
ts = TimeSeries(key=api_key, output_format='pandas')

# Initialize a DataFrame to store all the data
all_data_df = pd.DataFrame()

# Function to generate the past 12 months in 'YYYY-MM' format
def generate_past_12_months():
    return [(datetime.now() - timedelta(days=30*i)).strftime('%Y-%m') for i in range(0, 13)]

# Generate the list of the past 12 months
months = generate_past_12_months()

tickers = ['AAPL']  # Example ticker list

results = []

for ticker in tickers:
    print(f"Downloading daily data for {ticker} for the full history")
    try:
        # Get daily data
        daily_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
        daily_data.sort_index(ascending=True, inplace=True)
        daily_data.index = pd.to_datetime(daily_data.index)
    except Exception as e:
        print(f"Error for {ticker}: {e}")

    print(f"Downloading intraday data for {ticker} for the past year")

    # Calculate the slices for the past 12 months from the current date
    for month in months[::-1]:
        try:
            url = f'https://www.alphavantage.co/query?function=TIME_SERIES_INTRADAY&symbol={symbol}&interval={interval}&month={month}&outputsize={outputsize}&apikey={api_key}'
            response = requests.get(url)
            data = response.json()  # Data is in JSON format

            # The exact key for the time series data depends on the interval and function used
            # Adjust the key based on the actual structure of your JSON response
            time_series_key = f"Time Series ({interval})"

            if time_series_key in data:
                time_series_data = data["Time Series (5min)"]
                # Convert to DataFrame
                df = pd.DataFrame.from_dict(time_series_data, orient='index')
                df.index = pd.to_datetime(df.index)  # Converting index to datetime
                df.sort_index(inplace=True)  # Sorting by datetime index

                # Concatenate with the main DataFrame
                all_data_df = pd.concat([all_data_df, df])

            else:
                print(f"No data found for month: {month}")

            for day in pd.unique(df.index.date):
                day_str = day.strftime('%Y-%m-%d')
                day_data = all_data_df.between_time('09:30', '10:30').loc[day_str]
                if not day_data.empty:
                    open_price  = float(day_data.iloc[0]['1. open'])
                    close_price = float(day_data.iloc[-1]['4. close'])
                    high_price  = float(day_data['2. high'].max())
                    low_price   = float(day_data['3. low'].min())

                    # Find the last trading day's close price
                    today_open = daily_data.loc[day_str,'1. open']
                    previous_days_data = daily_data[daily_data.index < day_str]

                    # Calculate the gap percentage compared to yesterday's close
                    if not previous_days_data.empty:
                        previous_day_close = previous_days_data.iloc[-1]['4. close']
                        # print(type(previous_day_close), type(open_price))
                        gap_percentage = ((today_open - previous_day_close) / previous_day_close) * 100
                    else:
                        gap_percentage = None

                    results.append([day, ticker, open_price, close_price, high_price, low_price, gap_percentage])

        except Exception as e:
            print(f"Error for {ticker}: {e}")

# # # Convert the results into a DataFrame with an additional column for the gap percentage
results_df = pd.DataFrame(results, columns=['Date', 'Ticker', '9:30 Price', '10:30 Price', 'High Price', 'Low Price', 'Gap Percentage'])

results_df.to_csv('{ticker}.csv', index=False)


           Date Ticker  9:30 Price  10:30 Price  High Price  Low Price  \
0    2023-03-01   AAPL     123.200      122.981     123.764    122.689   
1    2023-03-02   AAPL     122.713      122.528     122.729    122.011   
2    2023-03-03   AAPL     123.630      123.455     123.752    123.014   
3    2023-03-06   AAPL     123.908      124.711     125.004    123.798   
4    2023-03-07   AAPL     124.519      124.028     124.670    123.511   
..          ...    ...         ...          ...         ...        ...   
253  2024-03-04   AAPL     187.760      192.385     192.440    187.600   
254  2024-03-05   AAPL     192.000      192.085     192.500    190.570   
255  2024-03-06   AAPL     193.500      195.240     195.250    192.960   
256  2024-03-07   AAPL     197.580      196.540     198.730    196.430   
257  2024-03-08   AAPL     196.060      197.150     197.360    195.620   

     Gap Percentage  
0         -0.393460  
1         -0.640011  
2          1.463231  
3          1.824141  
4

In [114]:
print(results_df)
results_df.to_csv('results_df.csv', index=False)

# daily_data, _ = ts.get_daily(symbol=ticker, outputsize='full')
# daily_data.sort_index(ascending=True, inplace=True)
# daily_data.index = pd.to_datetime(daily_data.index)
# daily_data.to_csv('daily_data.csv', index=True)

print(months)
# previous_days_data = daily_data[daily_data.index < '2023-03-02']
# print(previous_days_data)
# previous_day_close = previous_days_data.iloc[-1]['4. close']
# open_price  = float(daily_data.loc['2023-03-02', '1. open'])
# print(previous_day_close,open_price)

# for day in pd.unique(all_data_df.index.date):
#     day_str = day.strftime('%Y-%m-%d')

           Date Ticker  9:30 Price  10:30 Price  High Price  Low Price  \
0    2023-03-01   AAPL     123.200      122.981     123.764    122.689   
1    2023-03-02   AAPL     122.713      122.528     122.729    122.011   
2    2023-03-03   AAPL     123.630      123.455     123.752    123.014   
3    2023-03-06   AAPL     123.908      124.711     125.004    123.798   
4    2023-03-07   AAPL     124.519      124.028     124.670    123.511   
..          ...    ...         ...          ...         ...        ...   
253  2024-03-04   AAPL     187.760      192.385     192.440    187.600   
254  2024-03-05   AAPL     192.000      192.085     192.500    190.570   
255  2024-03-06   AAPL     193.500      195.240     195.250    192.960   
256  2024-03-07   AAPL     197.580      196.540     198.730    196.430   
257  2024-03-08   AAPL     196.060      197.150     197.360    195.620   

     Gap Percentage  
0         -0.393460  
1         -0.640011  
2          1.463231  
3          1.824141  
4